<a href="https://colab.research.google.com/github/jtkernan7/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/JT_Kernan_LS_DS_141_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None )

In [2]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
df[11][0] = 'n'


In [0]:
df = df.replace('?', None)
#I want to understand this code better
df = df.replace('n', 0)
df = df.replace('y', 1)
#yes = 1

In [24]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
2,democrat,0,1,1,1,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


In [0]:
dem = df.loc[df[0]=='democrat']
rep = df.loc[df[0]=='republican']

In [26]:
dem.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2,democrat,0,1,1,1,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1
5,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
6,democrat,0,1,0,1,1,1,0,0,0,0,0,0,1,1,1,1


In [27]:
rep.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
7,republican,0,1,0,1,1,1,0,0,0,0,0,0,1,1,1,1
8,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
10,republican,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,0


In [28]:
print(rep.shape, dem.shape)

(168, 17) (267, 17)


In [29]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
dtype: int64

In [32]:
dem.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000,267.000000
mean,0.599251,0.490637,0.872659,0.071161,0.224719,0.475655,0.767790,0.827715,0.734082,0.471910,0.494382,0.146067,0.295880,0.348315,0.625468,0.865169
std,0.490971,0.500851,0.333980,0.257576,0.418181,0.500345,0.423035,0.378337,0.442650,0.500148,0.500907,0.353837,0.457295,0.477331,0.484911,0.342185
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
rep.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,0.190476,0.482143,0.130952,0.988095,0.952381,0.898810,0.255952,0.166667,0.113095,0.547619,0.136905,0.827381,0.851190,0.958333,0.107143,0.666667
std,0.393851,0.501175,0.338357,0.108782,0.213596,0.302482,0.437700,0.373792,0.317656,0.499215,0.344774,0.379047,0.356965,0.200424,0.310219,0.472814
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df.shape

(435, 17)

In [38]:
rep.shape

(168, 17)

In [0]:
group2

In [40]:
group1 = rep[1]
group2 = dem[1]

stat, pvalue = ttest_ind(group1, group2)
print(pvalue <= 0.05)
print(stat, pvalue)

True
-9.103596896264648 3.229224805329127e-18


In [43]:
print(group1.std())
print(group2.std())

0.31021943722163997
0.48491072149138503


In [42]:
group1 = rep[15]
group2 = dem[15]

stat, pvalue = ttest_ind(group1, group2)
print(pvalue <= 0.05)
print(stat, pvalue)

True
-12.35237116310192 3.0884255458450424e-30


In [44]:
for sample in [group1, group2]:
    print(f"Mean: {sample.mean()}")
    print(f"Standard Deviation: {sample.std()}")
    print(f"Variance: {sample.var()}")
    print("---"*10)

Mean: 0.10714285714285714
Standard Deviation: 0.31021943722163997
Variance: 0.09623609923011102
------------------------------
Mean: 0.6254681647940075
Standard Deviation: 0.48491072149138503
Variance: 0.23513840781729556
------------------------------


In [45]:
group1 = rep[3]
group2 = dem[3]

stat, pvalue = ttest_ind(group1, group2)
print(pvalue <= 0.05)
print(stat, pvalue)

for sample in [group1, group2]:
    print(f"Mean: {sample.mean()}")
    print(f"Standard Deviation: {sample.std()}")
    print(f"Variance: {sample.var()}")
    print("---"*10)

True
-22.437726247750923 1.550290571202001e-74
Mean: 0.13095238095238096
Standard Deviation: 0.3383567866677383
Variance: 0.11448531508411734
------------------------------
Mean: 0.8726591760299626
Standard Deviation: 0.33398039193838625
Variance: 0.11154290219931809
------------------------------


In [46]:
group1 = rep[5]
group2 = dem[5]

stat, pvalue = ttest_ind(group1, group2)
print(pvalue <= 0.05)
print(stat, pvalue)

for sample in [group1, group2]:
    print(f"Mean: {sample.mean()}")
    print(f"Standard Deviation: {sample.std()}")
    print(f"Variance: {sample.var()}")
    print("---"*10)

True
20.897604777126915 1.435893650544929e-67
Mean: 0.9523809523809523
Standard Deviation: 0.21359550471631136
Variance: 0.045623039635015794
------------------------------
Mean: 0.2247191011235955
Standard Deviation: 0.4181810501732492
Variance: 0.17487539072400157
------------------------------


In [47]:
group1 = rep[2]
group2 = dem[2]

stat, pvalue = ttest_ind(group1, group2)
print(pvalue <= 0.05)
print(stat, pvalue)

for sample in [group1, group2]:
    print(f"Mean: {sample.mean()}")
    print(f"Standard Deviation: {sample.std()}")
    print(f"Variance: {sample.var()}")
    print("---"*10)

False
-0.17216828566462639 0.863385668888899
Mean: 0.48214285714285715
Standard Deviation: 0.5011748387445146
Variance: 0.2511762189905903
------------------------------
Mean: 0.49063670411985016
Standard Deviation: 0.5008511243103394
Variance: 0.25085184872293104
------------------------------


SO after working through tests i have determined that there are many issues that have statistical difference between republican and democrat.

issue 3, water project cost sharing, was way more supported by democrats as the numbers shows.

issue 5, physican fee-freeze, was way more supported by republicans. 

issue 2 had similar results for both republican and democrat and the t test caused us to accept the null hyptothesis. there are similarities with a p value of .863